# vworld api 사용해보기

https://datacook.tistory.com/11

open api를 사용하여 만들어진 사이트 : https://www.vworld.kr/dev/v4dv_apimash_s002.do?pageIndex=1&brdIde=21554&mashIde=s001&searchCondition=0&searchKeyword=%EC%A2%8C%ED%91%9C

In [1]:
from config import vworld_key #vworld 인증키 저장한 config.py 파일 불러오기

In [2]:
import pandas as pd
import json
import requests

In [3]:
df = pd.read_csv("파일이름.csv")

In [5]:
df.iloc[0][3] #위도

37.439

In [6]:
df.iloc[0][4] #경도

127.179

In [7]:
print(f'{df.iloc[0][4]},{df.iloc[0][3]}') #point test

127.179,37.439


In [8]:
print(type(f'{df.iloc[0][4]},{df.iloc[0][3]}'))

<class 'str'>


In [9]:
url = 'http://api.vworld.kr/req/address?' #서버 주소
params = 'service=address&request=getAddress&version=2.0&crs=epsg:4326&point='#파라미터
key = '&format=json&type=BOTH&zipcode=false&simple=true&key='#나머지 파라미터 및 key
my_key = vworld_key #config.py 파일로 설정
geocode = pd.DataFrame(columns = ['address','level4L','level4A'])

In [10]:
def request_geo(point):
    page = requests.get(url+params+point+key+my_key)
    json_data = json.loads(page.text)
    return json_data

In [26]:
def extraction_geo(test_data):
    geocode = pd.DataFrame(columns = ['address','level4L', 'level4A'])
    none = None
    for i in range(len(df)):
        point = f'{df.iloc[i][4]},{df.iloc[i][3]}'
        json_data = request_geo(point)

        if json_data['response']['status'] == 'NOT_FOUND' or json_data['response']['status'] == 'ERROR': #오류일 때
            geocode = geocode.append({'address':none,'level4L':none,'level4A':none}, ignore_index=True)
            continue

        address = json_data['response']['result'][0]['text']
        level4L = json_data['response']['result'][0]['structure']['level4L']
        level4A = json_data['response']['result'][0]['structure']['level4A']

        geocode = geocode.append({'address':address,'level4L':level4L, 'level4A':level4A}, ignore_index=True)
    return geocode

In [ ]:
#경도/위도 입력 test
json_data = request_geo(f'{df.iloc[0][4]},{df.iloc[0][3]}')

print(json_data)
print(type(json_data))

In [ ]:
#필요한 데이터 추출 test
address = json_data['response']['result'][0]['text']
level4L = json_data['response']['result'][0]['structure']['level4L']
level4A = json_data['response']['result'][0]['structure']['level4A']

print(address)
print(level4L)
print(level4A)

In [ ]:
df_address = extraction_geo(df) #함수 실행 

In [30]:
df_address.to_csv('저장.csv', index_label='NO' ,encoding='utf-8')